# Figures Three & Eight: MHD Model Results

- Top Panels: comparison of radial cuts of modeled observables with in-situ measurements
    - velocity
    - density
    - radial magnetic field
- Bottom Panel: comparison of MHD and PFSS footpoints
    - longitude vs. latitude plot
    - correlation plot

## Imports

In [ ]:
import os

import numpy as np
import pandas as pd
import scipy.stats as stats

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.ticker import MaxNLocator

import datetime
import astrospice

import tools.utilities as utils
from mhd.psipy.model import MASOutput
from tools.psp_funcs import ballistically_project

import astropy.units as u
import sunpy.coordinates as scoords
for sc in ['psp','solar orbiter'] : kernels = astrospice.registry.get_kernels(sc,'predict') 

# COLORS
c = ['#ae017e','#085A99',  '#c98000'] # darker colors
lightc = ['#FCA4C4',  '#8FD3F4', '#FFCC70'] # lighter colors
fcol = 'mistyrose'
sacol='lavender'
sacol = 'violet'
scol = 'lightgreen'
hcol = 'lightblue'
aa = 0.6
lw=2
clon = '#ae017e'
clat = '#085A99'
rcol = 'dimgrey'
cmaps = ['RdPu', 'cool', 'Wistia', 'spring']

# REGIONS
loc_hcs = [113, 116]
ssw = [166, 175]
sasw = [175, 185]
fsw = [70, 85]

# DIRECTORIES
IMG_DIR = './figures'
DF_DIR = './results'
PlotDir = '/Users/tamarervin/mplstyle/'

# PLOT STYLING
plot_style = os.path.join(PlotDir, 'figure_series.mplstyle')
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.cal'] = 'Helvetica Neue LT Pro'
plt.rcParams.update({'font.size': 18})
plt.style.use(plot_style)

## Data

In [ ]:
# regular data
parker = pd.read_csv('/Users/tamarervin/e11_conjunction/results/parker.csv')
orbiter = pd.read_csv('/Users/tamarervin/e11_conjunction/results/orbiter.csv')
abun = pd.read_csv('/Users/tamarervin/e11_conjunction/results/abun.csv')
smag = pd.read_csv('/Users/tamarervin/e11_conjunction/results/solo_mag.csv')
merged_df = pd.read_csv('/Users/tamarervin/e11_conjunction/results/merged_df.csv')
pss = pd.read_csv('/Users/tamarervin/e11_conjunction/results/pss.csv')

# timesampled data
parkerdownt = pd.read_csv('/Users/tamarervin/e11_conjunction/results/parkerdownt.csv')
orbiterdownt = pd.read_csv('/Users/tamarervin/e11_conjunction/results/orbiterdownt.csv')
abundownt = pd.read_csv('/Users/tamarervin/e11_conjunction/results/abundownt.csv')
smagdownt = pd.read_csv('/Users/tamarervin/e11_conjunction/results/smagdownt.csv')

# longitudinally sampled data
parkerdownl = pd.read_csv('/Users/tamarervin/e11_conjunction/results/parkerdownl.csv')
orbiterdownl = pd.read_csv('/Users/tamarervin/e11_conjunction/results/orbiterdownl.csv')
abundownl = pd.read_csv('/Users/tamarervin/e11_conjunction/results/abundownl.csv')
smagdownl = pd.read_csv('/Users/tamarervin/e11_conjunction/results/smagdownl.csv')

# PFSS/MHD data
tracerdf = pd.read_csv('/Users/tamarervin/e11_conjunction/mhd/mhd_footpoints.dat', sep='\s+') 
so_tracerdf = pd.read_csv('/Users/tamarervin/e11_conjunction/mhd/so_mhd_footpoints.dat', sep='\s+')
pfss = pd.read_csv('/Users/tamarervin/e11_conjunction/results/psp_pfss.csv')
so_pfss = pd.read_csv('/Users/tamarervin/e11_conjunction/results/so_pfss.csv')

# FIX TIMESTAMPS
parker['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in parker.Time]
orbiter['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S.%f') for d in orbiter.Time]
pss['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in pss.Time]
smag['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S.%f') for d in smag.Time]
abun['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S.%f') for d in abun.Time]
so_pfss['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in so_pfss.times]
parkerdownt['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in parkerdownt.Time]
orbiterdownt['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in orbiterdownt.Time]
smagdownt['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in smagdownt.Time]
abundownt['Time'] = [datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in abundownt.Time]


## Parker Trajectory

Functions from https://github.com/STBadman/PSP_E10_Sources/blob/main/helpers.py

In [ ]:
dt_start,dt_end = datetime.datetime(2022,2,18),datetime.datetime(2022,3,4)
dt_common = utils.gen_dt_arr(dt_start, dt_end, cadence_days=1/48)
ts_common = np.array([dt.timestamp() for dt in dt_common])

# get carrington locations at these timestamps for each instrument
### Create SkyCoord for PSP in the inertial (J2000) frame
psp_inertial = astrospice.generate_coords(
    'SOLAR PROBE PLUS', parker.Time

)
### Transform to solar co-rotating frame 
psp_carrington = psp_inertial.transform_to(
    scoords.HeliographicCarrington(observer="self")
)

# projection
psp_projected = ballistically_project(psp_carrington,vr_arr=parker.vr*u.km/u.s)

## MHD Plot

In [ ]:
# read in mhd data
corona_model = MASOutput("mhd/hmi_02_24_2022/") 
cbr=corona_model['br']
cbt=corona_model['bp']
cbn=corona_model['bt']
crho = corona_model['rho']
cvr = corona_model['vr']
cvt = corona_model['vp']
cvn = corona_model['vt']
cp = corona_model['p']
cjr = corona_model['jr']
cjt = corona_model['jp']
cjn = corona_model['jt']

In [ ]:
# resample the coordinates
rad = psp_carrington.radius
vr_sampled = cvr.sample_at_coords(psp_carrington.lon, psp_carrington.lat, rad)
rho_sampled = crho.radial_normalized(2).sample_at_coords(psp_carrington.lon, psp_carrington.lat, rad)
br_sampled = cbr.radial_normalized(2).sample_at_coords(psp_carrington.lon, psp_carrington.lat, rad)

In [ ]:
# plot in-situ data against the model
fig = plt.figure(figsize=(20, 14))
grid = plt.GridSpec(4, 2, height_ratios=[1, 1, 1, 2], width_ratios=[3, 1], wspace=0.1, hspace=0.25)

# create plots
ax1 = fig.add_subplot(grid[0, :])
ax2 = fig.add_subplot(grid[1, :])
ax3 = fig.add_subplot(grid[2, :])
ax4 = fig.add_subplot(grid[3, 0])
ax5 = fig.add_subplot(grid[3, 1])

# velocity
ax = ax1
ax.scatter(parker.lon, parker.vr, cmap='RdPu', c=parker.vr, s=3, label=r'$\rm Raw \; Data$')
ax.step(parkerdownl.lon, parkerdownl.vr, color='black', label=r'$\rm Binned \; Data$')
ax.plot(parker.lon, vr_sampled, color='red', label=r'$\rm MHD \; Results$')
ax.set(ylim=(125, 875), yticks=np.arange(200, 801, step=150))
ax.set_ylabel(ylabel=r'$\rm v_R \; [km \; s^{-1}]$', fontsize=18)
ax.set_xticklabels([])
ax.legend(fontsize=16, loc='upper left')
ax.axhline(500, color='k', linestyle='dashed')

# density
ax = ax2
ax.plot(parker.lon, rho_sampled*(1e-5), color='red')
ax.scatter(parker.lon, parker.pdens, cmap='cool', c=parker.pdens, s=3)
ax.step(parkerdownl.lon, parkerdownl.pdens, color='black')
ax.set(ylim=(-10, 90), yticks=np.arange(0, 81, step=20))
ax.set_xticklabels([])
ax.set_ylabel(r'$\rm n_p R^2 \; [cm^{-3}]$', fontsize=18)

# magnetic field
ax = ax3
ax.plot(parker.lon, br_sampled*2, color='red')
im = ax.scatter(parker.lon, parker.BrR2, cmap='Wistia', c=parker.BrR2, s=3)
ax.step(parkerdownl.lon, parkerdownl.BrR2, color='black')
ax.set(ylim=(-7.5, 7.5), yticks=np.arange(-6, 6.1, step=3))
ax.set_xlabel(xlabel=r'$\rm Source \; Surface \; Longitude \; [deg]$', fontsize=18)
ax.set_ylabel(ylabel=r'$\rm B_R R^2 \; [nT]$', fontsize=18)
ax.axhline(0, color='k', linestyle='dashed')

# Shade regions
panel_labels=['(a)', '(b)', '(c)']
for i, ax in enumerate([ax1, ax2, ax3]):
    ax.tick_params(axis='both', which='major', labelsize=16) 
    # shade interesting regions
    ax.axvspan(fsw[0], fsw[1], alpha=1, color=fcol, zorder=-3)
    ax.axvspan(sasw[0], sasw[1], alpha=aa, color=sacol, zorder=-3)
    ax.axvspan(loc_hcs[0], loc_hcs[1], alpha=aa, color=hcol, zorder=-3)
    ax.axvspan(ssw[0], ssw[1], alpha=aa, color=scol, zorder=-3)
    ax.axvline(loc_hcs[0], color='k', linestyle='dotted', zorder=5)
    ax.axvline(loc_hcs[1], color='k', linestyle='dotted', zorder=5)
    ax.axvline(ssw[0], color='k', linestyle='dotted', zorder=5)
    ax.axvline(ssw[1], color='k', linestyle='dotted', zorder=5)
    ax.axvline(fsw[0], color='k', linestyle='dotted', zorder=5)
    ax.axvline(fsw[1], color='k', linestyle='dotted', zorder=5)
    ax.axvline(sasw[0], color='k', linestyle='dotted', zorder=5)
    ax.axvline(sasw[1], color='k', linestyle='dotted', zorder=5)
    ax.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
    ax.text(0.95, 0.93, panel_labels[i], transform=ax.transAxes, fontsize=20, fontweight='bold', va='top', ha='left')

# scatter plot
mlat = 90 - tracerdf.t*u.rad.to(u.deg)
ax4.scatter(pfss.flon, pfss.flat, c='black', s=5, label=r'$\rm PFSS \; Results$', marker='D')
ax4.scatter(tracerdf.p*u.rad.to(u.deg), mlat, c='red', s=5, label=r'$\rm MHD \; Results$', marker='D')
ax4.set_xlabel(r'$\rm Heliographic \; Longitude \; [deg]$', fontsize=18)
ax4.set_ylabel(r'$\rm Heliographic \; Latitude \; [deg]$', fontsize=18)
ax4.legend(fontsize=16, loc='upper left')
ax4.set(ylim=(-37.5, 37.5), yticks=np.arange(-30, 31, step=15))
ax4.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
ax4.text(0.95, 0.93, '(d)', transform=ax4.transAxes, fontsize=20, fontweight='bold', va='top', ha='left')

# PARKER
mlat = 90 - tracerdf.t*u.rad.to(u.deg)
props = dict(boxstyle='round', facecolor='white', alpha=0.5)
corr = stats.spearmanr(pfss.flon, tracerdf.p*u.rad.to(u.deg))
corr2 = stats.spearmanr(pfss.flat, mlat)
ax5.scatter(pfss.flon, tracerdf.p*u.rad.to(u.deg), color='black', label=r'$\rm Longitude: $' + str(np.round(corr[0], 3)), s=5, marker='D')
ax5.scatter(pfss.flat, mlat, color='red', label=r'$\rm Latitude: $' + str(np.round(corr2[0], 3)), s=5, marker='D')
ax5.set_xlabel(r'$\rm PFSS \; Results$', fontsize=18)
ax5.set_ylabel(r'$\rm MHD \; Results$', fontsize=18)
ax5.legend(fontsize=16, loc='upper left')
ax5.grid(True, linestyle='--', linewidth=0.5, alpha=0.5)
ax5.text(0.88, 0.93, '(e)', transform=ax5.transAxes, fontsize=20, fontweight='bold', va='top', ha='left')

for ax in [ax1, ax2, ax3, ax4]:
    ax.set(xlim=(45, 205), xticks=np.arange(50, 205, step=25))

ax = ax1
ax.text(0.18, 0.96, r'$\rm FSW$', transform=ax.transAxes, fontsize=20, fontweight='bold', va='top', ha='left')
ax.text(0.415, 0.96, r'$\rm HCS$', transform=ax.transAxes, fontsize=20, fontweight='bold', va='top', ha='left')
ax.text(0.765, 0.96, r'$\rm SSW$', transform=ax.transAxes, fontsize=20, fontweight='bold', va='top', ha='left')
ax.text(0.82, 0.96, r'$\rm SASW$', transform=ax.transAxes, fontsize=20, fontweight='bold', va='top', ha='left')

### SAVE FIGURE
plt.savefig(os.path.realpath('figures/fig3.png'), bbox_inches='tight')
plt.savefig(os.path.realpath('eps_figures/fig3.eps'), bbox_inches='tight')

## MHD Appendix Figure
- three panels: radial cut of MHD observables compared with in-situ measurements
    - left: radial velocity vs. PSP velocity
    - middle: scaled density vs. PSP scaled density
    - right: scaled radial field vs. SWA/HIS carbon charge state ratio

In [ ]:
fig, axs = plt.subplots(1, 3, sharey='all', tight_layout=True, figsize=[25, 10])
cbar_kwargs = {"orientation": "horizontal"}
r_idx = -18
import astropy.units as u
var = ["vr", "rho", "br"]
titles = [r'$\rm v_r, 13.03 R_{\odot}$', r'$\rm \rho, 13.45 R_{\odot}$', r'$\rm B_r, 13.45 R_{\odot}$']
# find lowest lat value
lat_low = np.min(np.deg2rad(pfss.lat))
# polarities
neg = np.where(pfss.lon <= 113)[0]
pos = np.where(pfss.lon > 113)[0]

for i in range(0, 3):
    ax = axs[i]
    corona_model[var[i]].plot_radial_cut(r_idx, ax=ax, cbar_kwargs=cbar_kwargs)
    corona_model["br"].contour_radial_cut(r_idx, levels=[0], ax=ax, colors="black", zorder=1)
    ax.plot(np.deg2rad(pfss.lon[neg]), np.deg2rad(pfss.lat[neg]), color='blue', zorder=2)
    ax.plot(np.deg2rad(pfss.lon[pos]), np.deg2rad(pfss.lat[pos]), color='red', zorder=2)
    ax.set_xticks(np.deg2rad([0, 60, 120, 180, 240, 300, 360]))
    ax.set_yticks(np.deg2rad([-90, -45, 0, 45, 90]))
    ax.set_xticklabels([r'$\rm 0^{\circ}$', r'$\rm 60^{\circ}$', r'$\rm 120^{\circ}$', r'$\rm 180^{\circ}$',
                         r'$\rm 240^{\circ}$', r'$\rm 300^{\circ}$', r'$\rm 360^{\circ}$'])
    ax.set_yticklabels([r'$\rm -90^{\circ}$', r'$\rm -45^{\circ}$', r'$\rm 0^{\circ}$', 
                         r'$\rm 45^{\circ}$', r'$\rm 90^{\circ}$'])
    ax.set_title(titles[i])

# plot the data
ssize = 0.5
zord = 5
# velocity 
ax = axs[0]
vel = utils.renormalize_data(np.deg2rad(parker.vr), lat_low + np.deg2rad(5), lat_low + np.deg2rad(45))
axs[0].scatter(np.deg2rad(parker.lon), vel, c = vel, cmap='RdPu', s=ssize, zorder=zord, label=r'$\rm v_{sw} \; [km \; s^{-1}]$')
# ax.set_xticklabels([r'$\rm 0^{\circ}$', r'$\rm 60^{\circ}$', r'$\rm 120^{\circ}$', r'$\rm 180^{\circ}$',
#                          r'$\rm 240^{\circ}$', r'$\rm 300^{\circ}$', r'$\rm 360^{\circ}$'])
# ax.set_yticklabels([r'$\rm -90^{\circ}$', r'$\rm -45^{\circ}$', r'$\rm 0^{\circ}$', 
#                          r'$\rm 45^{\circ}$', r'$\rm 90^{\circ}$'])
axs[0].legend(loc='upper right')

# density
ax = axs[1]
rho = utils.renormalize_data(np.deg2rad(parker.pdens), lat_low + np.deg2rad(5), lat_low + np.deg2rad(45))
axs[1].scatter(np.deg2rad(parker.lon), rho, c = rho, cmap='cool', s=ssize, zorder=zord, label=r'$\rm n_p\, R^2 \; [cm^{-3}]$')
# ax.set_xticklabels([r'$\rm 0^{\circ}$', r'$\rm 60^{\circ}$', r'$\rm 120^{\circ}$', r'$\rm 180^{\circ}$',
#                          r'$\rm 240^{\circ}$', r'$\rm 300^{\circ}$', r'$\rm 360^{\circ}$'])
# ax.set_yticklabels([r'$\rm -90^{\circ}$', r'$\rm -45^{\circ}$', r'$\rm 0^{\circ}$', 
#                          r'$\rm 45^{\circ}$', r'$\rm 90^{\circ}$'])
axs[1].legend(loc='upper right')

# magnetic field
ax = axs[2]
car = utils.renormalize_data(np.deg2rad(abun.car), lat_low + np.deg2rad(5), lat_low + np.deg2rad(45))
axs[2].scatter(np.deg2rad(abun.lon)[630:], car[630:], c=car[630:], cmap='Wistia', s=ssize, zorder=zord, label = r'$\rm C^{6+}/C^{4+}$')
ax.set_xticklabels([r'$\rm 0^{\circ}$', r'$\rm 60^{\circ}$', r'$\rm 120^{\circ}$', r'$\rm 180^{\circ}$',
                         r'$\rm 240^{\circ}$', r'$\rm 300^{\circ}$', r'$\rm 360^{\circ}$'])
ax.set_yticklabels([r'$\rm -90^{\circ}$', r'$\rm -45^{\circ}$', r'$\rm 0^{\circ}$', 
                         r'$\rm 45^{\circ}$', r'$\rm 90^{\circ}$'])
legend = axs[2].legend(loc='upper right')

### SAVE FIGURES 
plt.savefig('figures/fig8.png', bbox_inches='tight')
plt.savefig('eps_figures/fig8.eps', bbox_inches='tight')